In [1]:
import numpy as np
LENGTH = 3

In [2]:
def play_game(p1, p2, env, draw=False):
    current_player = None
    while not env.game_over():
        #alternate between players
        #p1 always start first
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
 
        if draw == 1 and current_player == p1:
            env.draw_board()
        elif draw == 2 and current_player == p2:
            env.draw_board()
            
        #current_player         
        current_player.take_action(env)
        
        state = env.get_state()
        
        p1.update_state_history(state)
        p2.update_state_history(state)
        
    if draw:
        env.draw_board()
    p1.update(env)
    p2.update(env)

In [3]:
def get_state_hash_and_winner(env, i=0, j=0):
    results = []
    
    for v in (0, env.x, env.o):
        env.board[i, j] = v #if empty board it should already be zero.
        if j == 2: 
            #it reached the right side of the board.
            #increment i to move downwards and set j=0 to start from left side unless i=2
            if i == 2:
                #board is full collect results and return 
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.winner
                results.append((state, winner, ended))
            else:
                results+=get_state_hash_and_winner(env, i+1, j=0)
        else:
            results += get_state_hash_and_winner(env, i, j+1)
    return results
        

In [4]:
def initialV_x(env, state_winner_triples):
    '''
    Initialize state values as follows
    if x wins, V(s)=1
    if x loses or draws, V(s)=0
    otherwise, V(s)=0.5
    '''
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.x:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V

In [5]:
def initialV_o(env, state_winner_triples):
    '''
    Initialize state values as follows
    if o wins, V(s)=1
    if o loses or draws, V(s)=0
    otherwise, V(s)=0.5
    '''
    V = np.zeros(env.num_states)
    for state, winner, ended in state_winner_triples:
        if ended:
            if winner == env.o:
                v = 1
            else:
                v = 0
        else:
            v = 0.5
        V[state] = v
    return V

In [6]:
class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH, LENGTH))
        self.x = -1 #represents the player x on the board
        self.o = 1 #represents the player o on the board
        self.winner = None
        self.ended = False
        self.num_states = 3**(LENGTH*LENGTH)
        self.opp = {}
        self.opp[self.x] = self.o
        self.opp[self.o] = self.x
        
        
    def is_empty(self, i, j):
        return self.board[i, j] == 0
    
    def reward(self, symbol):
        '''No Reward untill game is over'''
        if not self.game_over():
            return 0
        #if we get here, game is over.
        #sym will be self.x or self.o
        elif self.winner == symbol:
            return 1
        elif self.winner == self.opp[symbol]:
            return 0.5
        else:
            return 0
    
    def get_state(self):
        #returns the current state represented as int for all prmutation states.
        #Number of states wil be 3^|Board Size|, since each state can survive three possible values.
        #Some States are not possible, all are X, but we ignore that 
        #This is like finding an integer representation of base3 number.
        
        k=0
        h=0
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i, j] == 0:
                    v = 0
                elif self.board[i, j] == self.x:
                    v = 1
                elif self.board[i, j] ==  self.o:
                    v = 2
                h += (3**k) * v
                k += 1
        return h
    
    def game_over(self, force_recalculate=False):
        
        if not force_recalculate and self.ended:
            return self.ended

        #check rows
        for i in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[i].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        #check columns 
        for j in range(LENGTH):
            for player in (self.x, self.o):
                if self.board[:, j].sum() == player * LENGTH:
                    self.winner = player
                    self.ended = True
                    return True

        for player in (self.x, self.o):
            #top left -> bottom right
            if self.board.trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True

            #bottom left -> top right
            if np.fliplr(self.board).trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True
  
            #check if draw
            if np.all((self.board==0) == False):
                #winner stays None
                self.winner = None
                self.ended = True
                return True

        #game is not over
        self.winner = None
        return False
    
    def draw_board(self):
        for i in range(LENGTH):
            print("-"*15)
            for j in range(LENGTH):
                print("   ", end='')
                if self.board[i, j] == self.x:
                    print("x", end='')
                elif self.board[i, j] == self.o:
                    print("o", end='')
                else:
                    print(" ", end='')
            print("")
        print("-"*15)

In [7]:
class Agent:
    def __init__(self, eps=0.1, alpha=0.1):
        self.eps = eps #probablity of choosing random action instead of greedy
        self.alpha = alpha #learning rate
        self.verbose = False
        self.state_history = []
    
    def setV(self, V):
        self.V = V
        
    def set_symbol(self, sym):
        self.sym = sym
        
    def set_verbose(self, v):
        self.verbose = v
        
    def reset_history(self):
        self.state_history = []
        
    def take_action(self, env):
        #choosing an action based on epsilon greedy strategy
        r = np.random.rand()
        best_state = None
        
        if r < self.eps:
            #take a random action:
            if self.verbose:
                print("Taking a Random Action")
            
            possible_moves = []
            
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        possible_moves.append((i, j))
            idx = np.random.choice(len(possible_moves))
            next_move = possible_moves[idx]
            
        else:
            pos2value = {}
            next_move = None
            best_value = -1
            for i in range(LENGTH):
                for j in range(LENGTH):
                    if env.is_empty(i, j):
                        env.board[i, j] = self.sym
                        state = env.get_state()
                        env.board[i, j] = 0
                        pos2value[(i, j)] = self.V[state]
                        if self.V[state] > best_value:
                            best_value = self.V[state]
                            best_state = state
                            next_move = (i, j)
                            
            # if verbose=True, draw the board with the values
            
            if self.verbose:
                print ("Taking a Greedy Action")
                for i in range(LENGTH):
                    print("-"*20)
                    for j in range(LENGTH):
                        print("  ", end='')
                        if env.is_empty(i, j):
                            print(f"{round(pos2value[(i,j)], 2)}", end='')
                        else:
                            if env.board[i, j] == env.x:
                                print("x  ", end='')
                            elif env.board[i, j] == env.o:
                                print("o  ", end='')
                            else:
                                print(" ", end='')
                    print("")
                print("-"*20)
             
        #make the move
        env.board[next_move[0], next_move[1]] = self.sym    
                
    def update_state_history(self, s):
        '''cannot put this in take action, because take action only happens once every other iteration for each player.
        but state history needs to updated for every iteration
        We can pass env and get env.get_state(), but we don't want to do this twice, so we calculate and pass it in.
        '''
        self.state_history.append(s)
        
    def update(self, env):
        '''Here we want to do this at the end of an episode. which is not true for all the algorithims we study.
        We want to backtrack over the states so that :
        V(prev_state) = V(prev_state) + alpha * (V(next_state) - V(prev_state))
        where V(next_state) =  reward if it is the most current state
        '''
        
        reward = env.reward(self.sym)
        target = reward
        for prev_state in reversed(self.state_history):
            value = self.V[prev_state] + self.alpha *(target - self.V[prev_state])
            self.V[prev_state] = value
            target = value
        self.reset_history()
                    

In [8]:
class Human:
    def __init__(self):
        pass
    
    def set_symbol(self, sym):
        self.sym = sym
        
    def take_action(self, env):
        while True:
            #break if we make a legal value
            move = input("Enter Cordinates i, j for next move: ")
            i, j = move.split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i, j):
                env.board[i, j] = self.sym
                break
    def update(self, env):
        pass
    
    def update_state_history(self, s):
        pass
            

In [9]:
# if __name__ == '__main__':
#train the agent
p1 = Agent()
p2 = Agent()

#set initial V for p1 and p2
env = Environment()
state_winner_triples = get_state_hash_and_winner(env)

Vx = initialV_x(env, state_winner_triples)
Vo = initialV_o(env, state_winner_triples)
p1.setV(Vx)
p2.setV(Vo)

#giving each player their symbol
p1.set_symbol(env.x)
p2.set_symbol(env.o)

T = 10000
for t in range(T):
    if t%500 == 0:
        print(t)
    play_game(p1, p2, Environment())

#Human playing starts
human = Human()


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [10]:
while True:
    

    player = int(input("Hi Bharat. For First player enter 1, else enter 2: "))

    if player == 1:
        human.set_symbol(env.x)
        p1.set_verbose(False)
        p2.set_verbose(True)
        play_game(human, p2, Environment(), 1)

    if player == 2:
        human.set_symbol(env.o)
        p1.set_verbose(True)
        p2.set_verbose(False)
        play_game(p1, human, Environment(), 2)
    #We have set agent as player p1 to check whether the agent would select the center as a starting move. 
    answer = input("Play again? [y/n]: ")

    if answer == 'n':
        break

Hi Bharat. For First player enter 1, else enter 2: 1
---------------
            
---------------
            
---------------
            
---------------
Enter Cordinates i, j for next move: 1,1
Taking a Greedy Action
--------------------
  0.45  0.47  0.49
--------------------
  0.47  x    0.47
--------------------
  0.48  0.42  0.48
--------------------
---------------
           o
---------------
       x    
---------------
            
---------------
Enter Cordinates i, j for next move: 2,2
Taking a Greedy Action
--------------------
  0.49  0.47  o  
--------------------
  0.47  x    0.48
--------------------
  0.41  0.47  x  
--------------------
---------------
   o       o
---------------
       x    
---------------
           x
---------------
Enter Cordinates i, j for next move: 0,1
Taking a Greedy Action
--------------------
  o    x    o  
--------------------
  0.42  x    0.46
--------------------
  0.46  0.49  x  
--------------------
---------------
   o   x   o
---